<a href="https://colab.research.google.com/github/Nishilhmodi/Fake_News_Prediction/blob/main/Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
news_dataset = pd.read_csv('/content/Fake_news_train.csv', encoding='latin-1')

<ipython-input-58-edd9a898943b>:1: DtypeWarning: Columns (0,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,26

In [ ]:
news_dataset.shape

(25116, 686)

In [ ]:
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 676,Unnamed: 677,Unnamed: 678,Unnamed: 679,Unnamed: 680,Unnamed: 681,Unnamed: 682,Unnamed: 683,Unnamed: 684,Unnamed: 685
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
news_dataset.isnull().sum()

,0
id,3
title,2250
author,4542
text,3079
label,3446
...,...
Unnamed: 681,25115
Unnamed: 682,25115
Unnamed: 683,25115
Unnamed: 684,25115


In [ ]:
news_dataset = news_dataset.fillna('')

In [ ]:
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
25111    Jerome Hudson Rapper T.I.: Trump a âPoster C...
25112    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
25113    Michael J. de la Merced and Rachel Abrams Macy...
25114    Alex Ansary NATO, Russia To Hold Parallel Exer...
25115              David Swanson What Keeps the F-35 Alive
Name: content, Length: 25116, dtype: object


In [ ]:
x = news_dataset.drop(columns='label', axis=1)
y = news_dataset['label']

In [ ]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
25111  20795  Rapper T.I.: Trump a âPoster Child For White...   
25112  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
25113  20797  Macyâs Is Said to Receive Takeover Approach ...   
25114  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
25115  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
25111    jerom hudson rapper trump poster child white s...
25112    benjamin hoffman n f l playoff schedul matchup...
25113    michael j de la merc rachel abram maci said re...
25114    alex ansari nato russia hold parallel exercis ...
25115                            david swanson keep f aliv
Name: content, Length: 25116, dtype: object


In [ ]:
x = news_dataset['content'].values
y = news_dataset['label'].values

In [ ]:
print(x)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(y)

['1' '0' '1' ... '0' '1' '1']


In [ ]:
y.shape

(25116,)

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [ ]:
print(x)

  (0, 294)	0.2711143595304882
  (0, 2676)	0.3695330951455916
  (0, 3190)	0.25021444167545076
  (0, 3884)	0.36190187718479516
  (0, 4092)	0.2719193039217898
  (0, 5380)	0.2242110980219714
  (0, 7617)	0.21896464665093704
  (0, 8387)	0.2514419730052186
  (0, 9387)	0.2931650914234994
  (0, 9694)	0.365543809573664
  (0, 14648)	0.24170537647240825
  (0, 17070)	0.2880854334390338
  (1, 1617)	0.28684095327432707
  (1, 2043)	0.16033000789330373
  (1, 2405)	0.3819555242659426
  (1, 3036)	0.19465794276348
  (1, 3850)	0.2655601620053947
  (1, 5954)	0.7141085899028713
  (1, 7407)	0.1937754459590677
  (1, 18279)	0.3003541536609626
  (2, 3174)	0.3289786239632894
  (2, 3349)	0.4735061475862881
  (2, 5829)	0.3938520403026429
  (2, 6467)	0.3373308801281171
  (2, 10451)	0.4757873319489284
  :	:
  (25113, 3932)	0.21371008348018833
  (25113, 7656)	0.2216005995344619
  (25113, 9104)	0.22557147358170038
  (25113, 9775)	0.3629705176720701
  (25113, 10332)	0.2978183783941726
  (25113, 10414)	0.1782507786420312

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=2)

In [ ]:
model = LogisticRegression()

In [ ]:
def is_numeric(val):
    try:
        float(val)
        return True
    except ValueError:
        return False

numeric_mask = np.vectorize(is_numeric)(y_train)
y_train = y_train[numeric_mask]
x_train = x_train[numeric_mask]

In [ ]:
# Fit the model to your training data before making predictions
model.fit(x_train, y_train)

# Now you can predict
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9925885755603759


In [ ]:
y_test = np.where(np.vectorize(is_numeric)(y_test), y_test, '0')

# Now, you can safely convert to int
y_test = y_test.astype(int)

# Now, calculate the accuracy
x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print('Accuracy score of the testing data : ', testing_data_accuracy)

Accuracy score of the testing data :  0.8077229299363057


In [ ]:
x_new = x_test[3]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(y_test[3])

1
